# Wilson's Morning Wake Up Playlist Generator, Feature Engineering

This notebook is broken down into the following tasks:

* Clean and pre-process the data.
* Standardization and normalization of numerical variables
* Define features for harmonic sequencing
* Create train/test `.csv` files that hold the relevant features and class labels for train/test data points

In the _next_ notebook, Train_Deploy , we will use the features and `.csv` files created in _this_ notebook to train an estimator in a SageMaker notebook instance.

In [62]:
# Import libraries
import pandas as pd
import numpy as np
import os

This Wilson Morning Wake Up dataset is summarized in a `.csv` file named `wmw_tracks.csv`, which we can read in using `pandas`. This was created in the Explore notebook.

In [33]:
# Read file
csv_file = 'data/wmw_tracks.csv'
wmw_df = pd.read_csv(csv_file)

wmw_df.head()

,volume,position,track_name,artist_name,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,38,1,Finding It There,Goldmund,0.187,0.00257,1,-37.134,1,0.0427,...,0.0915,0.0374,123.707,audio_features,6CnPCuUcM3A5PMP4gUy0vw,spotify:track:6CnPCuUcM3A5PMP4gUy0vw,https://api.spotify.com/v1/tracks/6CnPCuUcM3A5...,https://api.spotify.com/v1/audio-analysis/6CnP...,220120,5
1,38,2,Light Forms,Rohne,0.671,0.54500,10,-12.848,0,0.0393,...,0.1180,0.2840,133.036,audio_features,6MkUPsz5hYeneo0a9H0VT8,spotify:track:6MkUPsz5hYeneo0a9H0VT8,https://api.spotify.com/v1/tracks/6MkUPsz5hYen...,https://api.spotify.com/v1/audio-analysis/6MkU...,265870,4
2,38,3,C-Side,Khruangbin,0.688,0.77900,11,-10.129,0,0.0579,...,0.3490,0.9380,94.073,audio_features,6GvAM8oyVApQHGMgpBt8yl,spotify:track:6GvAM8oyVApQHGMgpBt8yl,https://api.spotify.com/v1/tracks/6GvAM8oyVApQ...,https://api.spotify.com/v1/audio-analysis/6GvA...,283407,4
3,38,4,Didn't I (Dave Allison Rework),Darondo,0.539,0.70500,0,-6.729,1,0.0527,...,0.1330,0.6850,186.033,audio_features,1owjOeZt1BdYWW6T8fIAEe,spotify:track:1owjOeZt1BdYWW6T8fIAEe,https://api.spotify.com/v1/tracks/1owjOeZt1BdY...,https://api.spotify.com/v1/audio-analysis/1owj...,328000,4
4,38,5,Woman Of The Ghetto - Akshin Alizadeh Remix,Marlena Shaw,0.707,0.57300,7,-8.403,0,0.0276,...,0.0858,0.1890,100.006,audio_features,2h8cQH7zhUWrynZi2MKhhC,spotify:track:2h8cQH7zhUWrynZi2MKhhC,https://api.spotify.com/v1/tracks/2h8cQH7zhUWr...,https://api.spotify.com/v1/audio-analysis/2h8c...,302467,4


In [34]:
# Show duplicated songs - woops!
wmw_df[wmw_df.duplicated(['track_name', 'artist_name'], keep=False)].sort_values('track_name')

,volume,position,track_name,artist_name,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
525,3,15,4AM,Vessels,0.482,0.794,0,-10.800,0,0.0442,...,0.1070,0.0382,150.036,audio_features,4MCL9tV2X5OiwT1oIl9pqK,spotify:track:4MCL9tV2X5OiwT1oIl9pqK,https://api.spotify.com/v1/tracks/4MCL9tV2X5Oi...,https://api.spotify.com/v1/audio-analysis/4MCL...,395229,4
451,8,15,4AM,Vessels,0.482,0.794,0,-10.800,0,0.0442,...,0.1070,0.0382,150.036,audio_features,77HWONgBl3FZbXarvXfRjc,spotify:track:77HWONgBl3FZbXarvXfRjc,https://api.spotify.com/v1/tracks/77HWONgBl3FZ...,https://api.spotify.com/v1/audio-analysis/77HW...,395229,4
192,26,10,Always Like This - Andhim Remix,HVOB,0.793,0.480,1,-12.183,0,0.0829,...,0.4690,0.2480,122.030,audio_features,7qfLlPC8tlohz2QYwGqkJW,spotify:track:7qfLlPC8tlohz2QYwGqkJW,https://api.spotify.com/v1/tracks/7qfLlPC8tloh...,https://api.spotify.com/v1/audio-analysis/7qfL...,359016,4
207,25,10,Always Like This - Andhim Remix,HVOB,0.793,0.480,1,-12.183,0,0.0829,...,0.4690,0.2480,122.030,audio_features,7qfLlPC8tlohz2QYwGqkJW,spotify:track:7qfLlPC8tlohz2QYwGqkJW,https://api.spotify.com/v1/tracks/7qfLlPC8tloh...,https://api.spotify.com/v1/audio-analysis/7qfL...,359016,4
184,26,2,Anchor (feat. Ishmael),Laurence Guy,0.706,0.640,11,-12.578,1,0.0416,...,0.0915,0.5750,121.024,audio_features,2FVgIrE55fd4ktm86pfhFx,spotify:track:2FVgIrE55fd4ktm86pfhFx,https://api.spotify.com/v1/tracks/2FVgIrE55fd4...,https://api.spotify.com/v1/audio-analysis/2FVg...,287666,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,34,6,U,DJ Seinfeld,0.721,0.838,8,-7.579,1,0.0428,...,0.0445,0.4530,125.999,audio_features,2tYctbOGPK2OZ2g2wKipgj,spotify:track:2tYctbOGPK2OZ2g2wKipgj,https://api.spotify.com/v1/tracks/2tYctbOGPK2O...,https://api.spotify.com/v1/audio-analysis/2tYc...,368000,4
339,15,8,Underworld,Edu Imbernon,0.878,0.482,8,-11.166,1,0.0483,...,0.0701,0.2970,123.023,audio_features,2gGeFW6SBAG5K5SNIQ4LW4,spotify:track:2gGeFW6SBAG5K5SNIQ4LW4,https://api.spotify.com/v1/tracks/2gGeFW6SBAG5...,https://api.spotify.com/v1/audio-analysis/2gGe...,425463,4
490,5,11,Underworld,Edu Imbernon,0.862,0.542,8,-10.615,1,0.0449,...,0.1070,0.3230,122.991,audio_features,2T0QyKXxEGsEGkY31deDam,spotify:track:2T0QyKXxEGsEGkY31deDam,https://api.spotify.com/v1/tracks/2T0QyKXxEGsE...,https://api.spotify.com/v1/audio-analysis/2T0Q...,320000,4
183,26,1,Vates,Otto Totland,0.360,0.231,4,-26.701,0,0.0336,...,0.0878,0.2820,104.094,audio_features,5lfBUkuT4mprXGQML18tyq,spotify:track:5lfBUkuT4mprXGQML18tyq,https://api.spotify.com/v1/tracks/5lfBUkuT4mpr...,https://api.spotify.com/v1/audio-analysis/5lfB...,256725,4


In [35]:
# Drop duplicates
print("Initial shape:", wmw_df.shape[0], 'tracks')
wmw_df.drop_duplicates(subset=['track_name', 'artist_name'], inplace=True)
print("New shape:", wmw_df.shape[0], 'tracks')

Initial shape: 554 tracks
New shape: 512 tracks


In [52]:
# Feature list
feature_list =  ['danceability','energy', 'loudness', 'speechiness', 'acousticness',
                 'instrumentalness', 'liveness', 'valence','mode','key','tempo']

processed_df = wmw_df.loc[:, feature_list]

processed_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,mode,key,tempo
0,0.187,0.00257,-37.134,0.0427,0.99400,0.952,0.0915,0.0374,1,1,123.707
1,0.671,0.54500,-12.848,0.0393,0.52400,0.853,0.1180,0.2840,0,10,133.036
2,0.688,0.77900,-10.129,0.0579,0.03870,0.118,0.3490,0.9380,0,11,94.073
3,0.539,0.70500,-6.729,0.0527,0.00828,0.135,0.1330,0.6850,1,0,186.033
4,0.707,0.57300,-8.403,0.0276,0.02500,0.291,0.0858,0.1890,0,7,100.006


In [53]:
from sklearn.preprocessing import StandardScaler

# Standardize features
standard_features = StandardScaler().fit_transform(processed_df)

processed_df = pd.DataFrame(standard_features, index=processed_df.index, columns=processed_df.columns)

processed_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,mode,key,tempo
0,-2.390995,-2.635095,-5.370442,-0.277322,2.096185,0.929695,-0.489837,-1.156919,1.085690,-1.204549,0.233803
1,0.291017,-0.131090,-0.375624,-0.352964,0.685856,0.645904,-0.283988,-0.125409,-0.921073,1.432297,0.682817
2,0.385220,0.949117,0.183583,0.060842,-0.770383,-1.461032,1.510395,2.610226,-0.921073,1.725280,-1.192510
3,-0.440441,0.607513,0.882849,-0.054846,-0.861664,-1.412300,-0.167469,1.551945,1.085690,-1.497532,3.233615
4,0.490506,-0.001835,0.538564,-0.613261,-0.811493,-0.965114,-0.534114,-0.522788,-0.921073,0.553349,-0.906949


In [61]:
processed_df.describe()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,mode,key,tempo
count,5.120000e+02,5.120000e+02,5.120000e+02,5.120000e+02,5.120000e+02,5.120000e+02,5.120000e+02,5.120000e+02,5.120000e+02,5.120000e+02,5.120000e+02
mean,2.389582e-16,4.597017e-17,5.833008e-17,8.364620e-17,9.280771e-17,-1.027824e-16,1.075529e-16,7.741204e-17,5.160802e-17,-1.778092e-17,-5.732177e-16
std,1.000978e+00,1.000978e+00,1.000978e+00,1.000978e+00,1.000978e+00,1.000978e+00,1.000978e+00,1.000978e+00,1.000978e+00,1.000978e+00,1.000978e+00
min,-3.070364e+00,-2.635095e+00,-5.491785e+00,-6.399586e-01,-8.864776e-01,-1.799288e+00,-9.955266e-01,-1.192893e+00,-9.210730e-01,-1.497532e+00,-3.116691e+00
25%,-4.071927e-01,-6.134908e-01,-3.255958e-01,-4.291622e-01,-8.346732e-01,-8.884327e-01,-5.088682e-01,-8.835651e-01,-9.210730e-01,-9.115661e-01,-3.627442e-01
50%,1.912728e-01,1.320371e-01,2.218371e-01,-2.773220e-01,-4.934187e-01,5.455737e-01,-3.538988e-01,-2.027936e-01,-9.210730e-01,-3.261724e-02,5.612778e-02
75%,7.024620e-01,7.183036e-01,6.001607e-01,7.447745e-03,6.768539e-01,7.670169e-01,-1.987943e-02,6.829414e-01,1.085690e+00,8.463316e-01,2.482667e-01
max,1.825970e+00,1.803127e+00,1.634972e+00,9.852028e+00,2.099185e+00,1.015693e+00,5.914790e+00,2.714799e+00,1.085690e+00,1.725280e+00,3.712663e+00


In [54]:
# Setup input and target
X = processed_df
y = processed_df.iloc[:,-3:]

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
def make_csv(x, y, filename, data_dir):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param y: Data labels
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''
    # make data dir, if it does not exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    
    # your code here
    data = pd.concat([pd.DataFrame(y), pd.DataFrame(x)], axis=1)
    data = data.dropna()
    data.to_csv(os.path.join(data_dir, filename), header=False, index=False)
    
    # nothing is returned, but a print statement indicates that the function has run
    print('Path created: '+str(data_dir)+'/'+str(filename))

In [60]:
# can change directory, if you want
data_dir = 'data'

make_csv(X_train, y_train, filename='train.csv', data_dir=data_dir)
make_csv(X_test, y_test, filename='test.csv', data_dir=data_dir)

Path created: data/train.csv
Path created: data/test.csv
